## Workup a03ws and c36m peptide sims with and without restraints 

Date: 4th November 2022 

In [ ]:
# Taken from Gabi's script 

## concatenate across all the full replica trajectories in order 

mkdir workup to work in 

# name each trajectory with replica number - only need this after time 
for i in {0..127}; do gmx_mpi trjcat -f ../r${i}/traj*.trr -nosort -cat -keeplast -o traj${i}_all_tmp.trr; done
for i in {0..47}; do gmx_mpi trjcat -f ../replica${i}/production_run*.trr -nosort -cat -keeplast -o traj${i}_all_tmp.trr; done

# writing index of atoms other than ions to concatentate trajectories as number of ions different in different replica simulations
for i in {0..47}; do echo -e '!15\nq' | gmx_mpi make_ndx -f ../replica${i}/production_run_input.tpr -o index${i}.ndx; done
for i in {1..47}; do echo 17 | gmx_mpi trjconv -f traj${i}_all_tmp.trr -o traj${i}_all_no_ions.trr -n index${i}.ndx; done
for i in {0..47}; do echo 1 | gmx_mpi trjconv -f traj${i}_all_tmp.trr -s ../replica${i}/production_run_input.tpr -o traj${i}_all_no_ions_water.trr; done


# join all trajectories together 
gmx_mpi trjcat -f traj?_all_tmp.trr traj??_all_tmp.trr traj???_all_tmp.trr -nosort -cat -keeplast -o traj_all.trr
gmx_mpi trjcat -f traj?_all_no_ions_water.trr traj??_all_no_ions_water.trr -nosort -cat -keeplast -o traj_all.trr

# fix timestep 
# why is this needed?
echo 0 | gmx_mpi trjconv -f traj_all.trr -t0 0 -timestep 1 -s ../r0/7-npt.tpr -o traj_all-skip-0.trr

# remove water 
echo 1 | gmx_mpi trjconv -f traj_all.trr  -t0 0 -timestep 1 -s ../r0/7-npt.tpr -o traj_all-skip-0-noW.trr
#timestep should be 200ps
echo 1 | gmx_mpi trjconv -f traj_all.trr  -t0 0 -timestep 1 -s ../replica0/production_run_input.tpr -o traj_all-skip-0-noW.trr

# remove individual trajectories 
for i in {0..127}; do rm -rf traj${i}_all_tmp.trr; done
for i in {0..47}; do rm -rf traj${i}_all_tmp.trr; done
# remove files that are not needed 
rm -rf traj_all.trr

#check number of frames so i know what mindist and weights should be 

gmx_mpi check -f traj_all-skip-0-noW.trr

#these are alice's notes
a03ws with cs = 518144 
a03ws without cs = 289024
c36m with cs = 534144
c36m without cs = 277120 


In [ ]:
## make whole
# what does this mean?

gmx_mpi trjconv -f traj_all-skip-0-noW.trr -s ../r0/7-npt.tpr -o confout.gro -b 0 -e 0 -pbc mol -center -ur compact
gmx_mpi trjconv -f traj_all-skip-0_200ps-noW.trr -s ../replica0/production_run_input.tpr -o confout.gro -b 0 -e 0 -pbc mol -center -ur compact

gmx_mpi trjconv -f traj_all-skip-0-noW.trr -s ../r0/7-npt.tpr -o traj_all-skip-0-noW-whole.trr -pbc mol -center -ur compact
gmx_mpi trjconv -f traj_all-skip-0_200ps-noW.trr -s ../replica0/production_run_input.tpr -o traj_all-skip-0-noW-whole-200ps.trr -pbc mol -center -ur compact
gmx_mpi trjconv -f traj_all-skip-0-noW.trr -s ../replica0/production_run_input.tpr -o traj_all-skip-0-noW-whole.trr -pbc mol -center -ur compact

In [ ]:
# find out self-interacting conformations as we don't want to include them

echo 1 | gmx_mpi mindist -f traj_all-skip-0-noW-whole-200ps.trr -s ../replica0/production_run_input.tpr -pi

tail -n +30 mindist.xvg | awk '{print $2}' > mindist_all.dat; rm -rf mindist.xvg


The shortest periodic distance is 0.144525 (nm) at time 8.6222e+06 (ps),
between atoms 36 and 435

a03ws with cs = The shortest periodic distance is 0.14305 (nm) at time 113603 (ps),
between atoms 15 and 339

a03ws without cs = The shortest periodic distance is 0.146889 (nm) at time 47842 (ps),
between atoms 15 and 339

c36m with cs = The shortest periodic distance is 0.146889 (nm) at time 47842 (ps),
between atoms 15 and 339

c6m without cs = The shortest periodic distance is 0.422048 (nm) at time 71951 (ps),
between atoms 3 and 332



In [ ]:
# run the driver on the full trajectory 
# need restart at the top of the plumed.dat
# backup HILLS files with date 

plumed driver --mf_trr traj_all-skip-0-noW-whole-200ps.trr --plumed plumed_driver.dat
plumed driver --mf_trr traj_all-skip-0-noW-whole.trr --plumed plumed_driver.dat

#using the final column --> output arguments pbmetad.bias 

submission script: 
    
#!/bin/bash 

#SBATCH --job-name=c36m-peptide
#SBATCH --nodes=32
#SBATCH --tasks-per-node=2
#SBATCH --cpus-per-task=64
#SBATCH --time=48:00:00
#SBATCH --mail-user=alice.pettitt.16@ucl.ac.uk

# Replace [budget code] below with your project code (e.g. t01)
#SBATCH --account=e692
#SBATCH --partition=standard
#SBATCH --qos=long

instead try 
64
1
128
128

this is still too long for a03ws and c36m with cs 

skip 2 

In [ ]:
this is still too long for a03ws and c36m with cs
try using skip 2 

gmx_mpi trjconv -f traj_all-skip-0-noW.trr -s ../r0/7-npt.tpr -o traj_all-skip-2-noW-whole.trr -pbc mol -center -ur compact -skip 2

echo 1 | gmx_mpi mindist -f traj_all-skip-2-noW-whole.trr -s ../r0/7-npt.tpr -pi

tail -n +30 mindist.xvg | awk '{print $2}' > mindist_all.dat; rm -rf mindist.xvg

a03ws with cs = The shortest periodic distance is 0.155184 (nm) at time 113520 (ps),
between atoms 15 and 339
c36m with cs = The shortest periodic distance is 0.317382 (nm) at time 330384 (ps),
between atoms 3 and 333

a03ws with cs = 259072
c36m with cs = 267072 

plumed_driver_2.dat and srun_2.sh for the skip 2 trajectory 


In [ ]:
08.11.22

skip 2 might also be too long, trying skip 3 

gmx_mpi trjconv -f traj_all-skip-0-noW.trr -s ../r0/7-npt.tpr -o ../skip3_workup_04_11_2022/traj_all-skip-3-noW-whole.trr -pbc mol -center -ur compact -skip 3

echo 1 | gmx_mpi mindist -f traj_all-skip-3-noW-whole.trr -s ../r0/7-npt.tpr -pi

tail -n +30 mindist.xvg | awk '{print $2}' > mindist_all.dat; rm -rf mindist.xvg

a03ws with cs = The shortest periodic distance is 0.149845 (nm) at time 113607 (ps),
between atoms 15 and 336

c36m with cs = The shortest periodic distance is 0.230231 (nm) at time 486843 (ps),
between atoms 2 and 333

gmx_mpi check 

a03ws with cs = 172715
c36m with cs = 178048

run plumed driver for skip 3 trajectory

In [ ]:
skip 3 worked 

11th November 2022 

In [ ]:
# remove first 5 lines from COLVAR
#ALI-- check how many lines you actually ened to remove in your case--might be different.
awk '{if($1!="#!") print}' COLVAR > COLVAR_nohead

# calculate weights - need to work out kbT at 310 K - double check calculation and row it uses 
# Ali--you need to recalculate your kbT
maxV=`awk '{print $NF}' COLVAR_nohead | sort -n | tail -n 1`
awk '{print exp(($NF-maxV)/kbT)}' maxV=$maxV kbT=2.57748 COLVAR_nohead > weights.dat
awk '{print exp(($NF-maxV)/kbT)}' maxV=$maxV kbT=2.31142 COLVAR_nohead > weights.dat


# check length of mindist_all and weights 
wc -l *

In [ ]:
# mindist 1.2 - for a03ws/a99sb
# Ali-- check the cutoffs in your mdp file. this is forcefield dependent!
paste weights.dat mindist_all.dat | awk '{w=$1; if($2<=mdcut) w=0.0; print w}' mdcut=1.2 > weights_corr.dat
w_tot=`awk '{tot+=$1}END{print tot}' weights.dat`
awk '{tot+=$1}END{print "KEPT PERCENTAGE",tot/w_tot*100}' w_tot=$w_tot weights_corr.dat

# KEPT PERCENTAGE 99.9997

# mindist 0.95 - for c36m 
# Ali-- check the cutoffs in your mdp file. this is forcefield dependent!

paste weights.dat mindist_all.dat | awk '{w=$1; if($2<=mdcut) w=0.0; print w}' mdcut=0.95 > weights_corr.dat
w_tot=`awk '{tot+=$1}END{print tot}' weights.dat`
awk '{tot+=$1}END{print "KEPT PERCENTAGE",tot/w_tot*100}' w_tot=$w_tot weights_corr.dat

MINDIST IS GOOD 

a03ws with cs:99.9991
a03ws without cs: 99.9991
c36m with cs: 100
c36m without cs: KEPT PERCENTAGE 100

In [ ]:
#can ignore if no chemical shifts.
camshift: 
    
RESTART
  
MOLINFO STRUCTURE=../system/template_md.pdb
prot: GROUP ATOMS=1-339
WHOLEMOLECULES ENTITY0=prot

# Chemical shifts at 310 K
cs: CS2BACKBONE ATOMS=7-339 DATADIR=../data NOPBC TEMPLATE=template_cs.pdb TEMP=310.0

PRINT ARG=cs.* FILE=CS_COLVAR STRIDE=1
     
plumed driver --mf_trr traj_all-skip-0-noW-whole.trr --plumed camshift_plumed.dat

a03ws and c36m with cs 
plumed driver --mf_trr traj_all-skip-3-noW-whole.trr --plumed camshift_plumed.dat

In [ ]:
should not have done skip 3 - need to be able to divide it by the 128 replicas 

## 13th November 2022 

In [ ]:
trying skip 4 

gmx_mpi trjconv -f traj_all-skip-0-noW.trr -s ../r0/7-npt.tpr -o ../skip4_workup_04_11_2022/traj_all-skip-4-noW-whole.trr -pbc mol -center -ur compact -skip 4

echo 1 | gmx_mpi mindist -f traj_all-skip-4-noW-whole.trr -s ../r0/7-npt.tpr -pi

tail -n +30 mindist.xvg | awk '{print $2}' > mindist_all.dat; rm -rf mindist.xvg

a03ws with cs = The shortest periodic distance is 0.155184 (nm) at time 113520 (ps),
between atoms 15 and 339

c36m with cs = The shortest periodic distance is 0.317382 (nm) at time 330384 (ps),
between atoms 3 and 333

PARK THIS FOR NOW - CHECK WITH ARCHER2 IF 60 HOUR TIME-LIMIT CAN BE USED 

gmx_mpi check 

a03ws with cs =
c36m with cs = 

run plumed driver for skip 4 trajectory

## 17th January 2023 - workup for a03ws and c36m with cs 

In [ ]:
going to try skip 4 for a03ws and c36m with cs 

working in 7-npt-mm-3fs/skip4_workup_04_11_2022

a03ws with cs = 129536
c36m with cs = 133536 --> not divisble by 128


In [ ]:
STOP HERE

In [ ]:
# remove first 5 lines from COLVAR
awk '{if($1!="#!") print}' COLVAR > COLVAR_nohead

# calculate weights - need to work out kbT at 310 K - double check calculation and row it uses 

maxV=`awk '{print $NF}' COLVAR_nohead | sort -n | tail -n 1`
awk '{print exp(($NF-maxV)/kbT)}' maxV=$maxV kbT=2.57748 COLVAR_nohead > weights.dat

# check length of mindist_all and weights 
wc -l *

In [ ]:
# mindist 1.2 - for a03ws/a99sb
paste weights.dat mindist_all.dat | awk '{w=$1; if($2<=mdcut) w=0.0; print w}' mdcut=1.2 > weights_corr.dat
w_tot=`awk '{tot+=$1}END{print tot}' weights.dat`
awk '{tot+=$1}END{print "KEPT PERCENTAGE",tot/w_tot*100}' w_tot=$w_tot weights_corr.dat

# mindist 0.95 - for c36m 

paste weights.dat mindist_all.dat | awk '{w=$1; if($2<=mdcut) w=0.0; print w}' mdcut=0.95 > weights_corr.dat
w_tot=`awk '{tot+=$1}END{print tot}' weights.dat`
awk '{tot+=$1}END{print "KEPT PERCENTAGE",tot/w_tot*100}' w_tot=$w_tot weights_corr.dat

MINDIST IS GOOD 

a03ws with cs:99.9991
a03ws without cs: 99.9991
c36m with cs: 100
c36m without cs: KEPT PERCENTAGE 100

In [ ]:
camshift: 
    
RESTART
  
MOLINFO STRUCTURE=../system/template_md.pdb
prot: GROUP ATOMS=1-339
WHOLEMOLECULES ENTITY0=prot

# Chemical shifts at 310 K
cs: CS2BACKBONE ATOMS=7-339 DATADIR=../data NOPBC TEMPLATE=template_cs.pdb TEMP=310.0

PRINT ARG=cs.* FILE=CS_COLVAR STRIDE=1
     
plumed driver --mf_trr traj_all-skip-0-noW-whole.trr --plumed camshift_plumed.dat

a03ws and c36m with cs 
plumed driver --mf_trr traj_all-skip-4-noW-whole.trr --plumed camshift_plumed.dat